## Back Test Data Generation

In [ ]:
import requests
import time
import calendar
import dateutil.parser as parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import yaml
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from pycaret.classification import *

In [ ]:
now = datetime.now()
today = now.strftime("%d-%m-%Y_%I-%M_%p")

In [ ]:
with open ('back_test_pipeline_settings.yaml') as ymlfile:
    cfg = yaml.safe_load(ymlfile)
    k_number = cfg['knn']['k_number']
    metric = cfg['knn']['metric']
    algorithm = cfg['knn']['algorithm']
    feature_1 = cfg['knn']['feature_1']
    feature_2 = cfg['knn']['feature_2']
    feature_3 = cfg['knn']['feature_3']
    feature_7 = cfg['knn']['feature_7']
    feature_8 = cfg['knn']['feature_8']
    feature_15 = cfg['knn']['feature_15']
    volume = cfg['feature']['volume']
    volume_size = cfg['sample']['volume_size']
    sample_count = cfg['sample']['count']
    candles = cfg['recommendation']['candle_count']
    pair = cfg['currency']['pair']
    instrument = cfg['currency']['instrument']
    model = cfg['model']['name']

In [ ]:
print('K Number:',k_number)
print('Metric:', metric)
print('Algorithm:', algorithm)
print('Candle Volume Size:', volume_size)
print('Random Sample Count:', sample_count)
print('Future Candle Count:', candles)
print('Pair:', pair)
print('Instrument:', instrument)
print('Model:', model)

In [ ]:
def convert_date(utc_time): 
    parsed_date = parser.parse(utc_time)
    var_date=parsed_date.date()
    var_time=parsed_date.time()
    var_f_time=var_time.hour
    var_julian_date=parsed_date.timetuple().tm_yday
    var_weekday=parsed_date.weekday()
    var_weekday_name=calendar.day_name[parsed_date.weekday()]
    return var_date, var_time, var_f_time, var_julian_date, var_weekday, var_weekday_name

In [ ]:
def find_k_similar_candles(candle_id, dataset, k=k_number):
    indices=[]
    distances = []
    output = []
    model_knn = NearestNeighbors(metric = metric, algorithm = algorithm) 
    model_knn.fit(dataset)
    
    #metric = 'euclidean' or 'cosine' or 'manhattan' or 'mahalanobis'
    
    distances, indices = model_knn.kneighbors(dataset.iloc[candle_id,:].values.reshape(1,-1),
                                              n_neighbors = k)

    for i in range(0,len(distances.flatten())):
        if i!=0:
            
            output.append ([dataset.index[indices.flatten()[i]],
                            distances.flatten()[i],
                            dataset.iloc[indices.flatten()[i]][feature_1],
                            dataset.iloc[indices.flatten()[i]][feature_2],
                            dataset.iloc[indices.flatten()[i]][feature_3],
                            dataset.iloc[indices.flatten()[i]][feature_7],
                            dataset.iloc[indices.flatten()[i]][feature_8],
                           ])
    
    output = pd.DataFrame(output)
    output.columns = ['Indice','Distance',
                      feature_1,
                      feature_2,
                      feature_3,
                      feature_7,
                      feature_8,
                     ]
   # display (output)
    
    return indices, distances

# <font color='red'>Test Configs</font>

In [ ]:
filename = '{}_H4.csv'.format(instrument)
data = pd.read_csv(filename)

In [ ]:
data.columns

In [ ]:
data.head()

## Selecting 1000 Random Candles

In [ ]:
random_samples = data[data[volume] > volume_size].sample(n = sample_count)
Test_Candle = list(random_samples.index.values)

In [ ]:
Test_Candle[0:10]

In [ ]:
data.head()

# <font color='red'>CANDLE LOOP</font>

In [ ]:
FINAL_MODEL = load_model('USDCHF/KNN/02/14-03-2022_03-38_AM_knn_USDCHF')

In [ ]:
%%time
print ('Today: ' + today)

result_output = pd.DataFrame({'Candle_No':[],
                              'Current_Market_Fit':[],
                              'Current_Market':[],
                              '{}_Label'.format(model):[],
                              '{}_Score'.format(model):[],                            
                             })

for candle_no in Test_Candle:
    data = pd.read_csv(filename)
    data = data.iloc[candle_no:candle_no+candles]
    data['candleno'] = range (1, len(data) + 1)
    X = data['candleno'].values.reshape(-1, 1)
    Y = data['Close'].values.reshape(-1, 1)
    linear_regressor = LinearRegression()
    linear_regressor.fit(X, Y)
    y_pred = linear_regressor.predict(X) 
    
    Current_Market_Fit = r2_score(Y, y_pred)*100
    coeficient = (linear_regressor.coef_)

    if coeficient > 0:
        Current_Market= 1

    else:
        Current_Market = 0

    data = pd.read_csv(filename)
    data = data[[feature_1,
                 feature_2,
                 feature_3,
                 feature_7,
                 feature_8,
                ]]

    indices, distances = find_k_similar_candles (candle_no,data)
    indices = indices[0:1][0]
    
    L_L = []
    H_H = []
    predicted_output_1 = []
    predicted_output_2 = []
    predicted_output_3 = []
        
    for indice in indices[1:5]:
        #print(indice)
        data = pd.read_csv(filename) 
        data = data.iloc[indice:indice+candles]
        
        HH = data.iloc[0]['Close'] - data['High'].max()
        LL = data.iloc[0]['Close'] - data['Low'].min()
        
        L_L.append([LL])
        H_H.append([HH])

        data['candleno'] = range (1, len(data) + 1)
        X = data['candleno'].values.reshape(-1, 1)
        
        Y_Close = data['Close'].values.reshape(-1, 1)
        linear_regressor = LinearRegression()
        linear_regressor.fit(X, Y_Close)
        y_pred = linear_regressor.predict(X)
        Predicted_Market_Fit_Close = r2_score(Y_Close, y_pred)*100
        coeficient_close = (linear_regressor.coef_)
        if coeficient_close > 0:
            Predicted_Trade_Close = 1    ## Buy ##
        else:
            Predicted_Trade_Close = -1   ## Sell ##
        
        predicted_output_1.append([Predicted_Market_Fit_Close * Predicted_Trade_Close])
        
        
        Y_High = data['High'].values.reshape(-1, 1)
        linear_regressor.fit(X, Y_High)
        y_pred = linear_regressor.predict(X)
        Predicted_Market_Fit_High= r2_score(Y_High, y_pred)*100
        coeficient_high = (linear_regressor.coef_)
        
        if coeficient_high > 0:
            Predicted_Trade_High = 1    ## Buy ##
        else:
            Predicted_Trade_High = -1   ## Sell ##            
        
        predicted_output_2.append([Predicted_Market_Fit_High * Predicted_Trade_High])
        
        
        Y_Low = data['Low'].values.reshape(-1, 1)
        linear_regressor.fit(X, Y_Low)
        y_pred = linear_regressor.predict(X)
        Predicted_Market_Fit_Low= r2_score(Y_Low, y_pred)*100
        coeficient_low = (linear_regressor.coef_)
        if coeficient_low > 0:
            Predicted_Trade_Low = 1    ## Buy ##
        else:
            Predicted_Trade_Low = -1   ## Sell ##
        
        predicted_output_3.append([Predicted_Market_Fit_Low * Predicted_Trade_Low])
        
        
    data_unseen = pd.DataFrame ({
        'Rec1_Close_Score': [predicted_output_1[0][0]],
#        'Rec1_High_Score': [predicted_output_2[0][0]],
#        'Rec1_Low_Score': [predicted_output_3[0][0]],
#        'Rec1_HH': [H_H[0][0]],
#        'Rec1_LL': [L_L[0][0]],
              
        'Rec2_Close_Score': [predicted_output_1[1][0]],
#        'Rec2_High_Score': [predicted_output_2[1][0]],
#        'Rec2_Low_Score': [predicted_output_3[1][0]],
#        'Rec2_HH': [H_H[1][0]],
#        'Rec2_LL': [L_L[1][0]],
              
        'Rec3_Close_Score': [predicted_output_1[2][0]],
#        'Rec3_High_Score': [predicted_output_2[2][0]],
#        'Rec3_Low_Score': [predicted_output_3[2][0]],
#        'Rec3_HH': [H_H[2][0]],
#        'Rec3_LL': [L_L[2][0]],
              
        'Rec4_Close_Score': [predicted_output_1[3][0]],
#        'Rec4_High_Score': [predicted_output_2[3][0]],
#        'Rec4_Low_Score': [predicted_output_3[3][0]],
#        'Rec4_HH': [H_H[3][0]],
#        'Rec4_LL': [L_L[3][0]]
    })
        
    model_prediction = predict_model(FINAL_MODEL, data=data_unseen)
    MODEL_Label = model_prediction['Label']
    MODEL_Score = model_prediction['Score']
  
    result = {'Candle_No': candle_no,
              'Current_Market_Fit': Current_Market_Fit,
              'Current_Market': Current_Market,
              '{}_Label'.format(model): MODEL_Label[0],
              '{}_Score'.format(model): MODEL_Score[0],
             }
    
    result_output = result_output.append(result, ignore_index = True)

In [ ]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [ ]:
result_output = result_output[result_output['Current_Market_Fit'] > 16]
result_output = result_output.reset_index()
del result_output['index']

In [ ]:
result_output.shape

In [ ]:
result_output.to_csv('03_Back_Test_Final_Result_' + today + '.csv', header = True, index = False)
result_output = pd.read_csv('03_Back_Test_Final_Result_' + today + '.csv')

In [ ]:
result_output['MODEL_Prediction'] = result_output['Current_Market'] - result_output['{}_Label'.format(model)]

In [ ]:
result_output.head(30)

In [ ]:
MODEL_Result = pd.DataFrame(result_output['MODEL_Prediction'].value_counts())
MODEL_Result["Score"] = (MODEL_Result['MODEL_Prediction'] * 100 / len(result_output)).round(2)

In [ ]:
result = pd.concat([
    MODEL_Result,
], axis=1)
result

In [ ]:
file = open(today + "_" + model + "_final_test_log_" + pair + ".txt", "w")
file.write ("Date: " + today + "\n" + \
            "Currency Pair: " + pair + "\n" + \
            "Model: " + model + "\n" + \
            "Model Input: " + str(data_unseen.columns) + "\n" + \
            "Final Result: " + str(result) + "\n"
           )
file.close()